In [3]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pyspark
from pyspark.sql import SparkSession
import logging
from datetime import datetime
from pyspark.sql.functions import lit, coalesce, col, current_date
from configs import configs
from functions import functions as F
from dotenv import load_dotenv
import os

In [5]:
load_dotenv()

HOST_ADDRESS=os.getenv('HOST_ADDRESS')
MINIO_ACCESS_KEY=os.getenv('MINIO_ACCESS_KEY')
MINIO_SECRET_KEY=os.getenv('MINIO_SECRET_KEY')

In [6]:
if __name__ == "__main__":
    spark = SparkSession.builder \
            .appName("process_bronze_to_silver_isp_performance") \
            .config("spark.hadoop.fs.s3a.endpoint", f"http://{HOST_ADDRESS}:9000") \
            .config("spark.hadoop.fs.s3a.access.key", MINIO_ACCESS_KEY) \
            .config("spark.hadoop.fs.s3a.secret.key", MINIO_SECRET_KEY) \
            .config("spark.hadoop.fs.s3a.path.style.access", True) \
            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
            .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
            .config("hive.metastore.uris", "thrift://metastore:9083") \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
            .config("spark.executor.memory", "4g") \
            .config("spark.driver.memory", "4g") \
            .config("spark.memory.fraction", "0.8") \
            .config("spark.sql.shuffle.partitions", "50") \
            .getOrCreate()

In [ ]:
df = spark.read.format("parquet").load('s3a://landing/isp_performance/landing_ordem_servico_aberto').show(100)

In [7]:
query = """
WITH BASE_PERFORMANCE AS (
    SELECT
        t1.ano_abertura,
        t1.ano_mes_abertura,
        t1.data_abertura,
        t1.ano_fechamento,
        t1.ano_mes_fechamento,
        t1.data_fechamento,
        t2.id AS id_filial,
        t2.fantasia AS filial,
        t4.id AS id_setor,
        t4.setor AS setor,
        t6.id AS id_relator,
        t6.login AS relator,
        t5.id AS id_tecnico,
        t5.funcionario AS tecnico,
        t3.id AS id_assunto,
        t3.assunto AS assunto,
        t1.id AS ordem_servico_id,
        t1.id_status,
        t1.status,
        t1.id_status_sla,
        t1.status_sla
    FROM
        silver_ordem_servico t1
    LEFT JOIN silver_dim_filial t2 ON (t2.id = t1.id_filial)
    LEFT JOIN silver_dim_assunto t3 ON (t3.id = t1.id_assunto)
    LEFT JOIN silver_dim_setor t4 ON (t4.id = t1.id_setor)
    LEFT JOIN silver_dim_colaboradores t5 ON (t5.id = t1.id_tecnico)
    LEFT JOIN silver_dim_usuarios t6 ON (t6.id = t1.id_login)
    ORDER BY
        t1.ano_mes_abertura,
        t2.id
),
STATUS_COUNTS AS (
    SELECT
        ano_abertura,
        ano_mes_abertura,
        ano_fechamento,
        ano_mes_fechamento,
        id_filial,
        filial,
        id_setor,
        setor,
        id_assunto,
        assunto,
        id_relator,
        relator,
        id_tecnico,
        tecnico,
        id_status,
        status,
        data_abertura,
        data_fechamento,
        id_status_sla,
        status_sla,
        COUNT(ordem_servico_id) AS qtd
    FROM BASE_PERFORMANCE
    GROUP BY
        ano_abertura,
        ano_mes_abertura,
        ano_fechamento,
        ano_mes_fechamento,
        id_filial,
        filial,
        id_setor,
        setor,
        id_assunto,
        assunto,
        id_relator,
        relator,
        id_tecnico,
        tecnico,
        id_status,
        status,
        data_abertura,
        data_fechamento,
        id_status_sla,
        status_sla
)
SELECT
    ano_abertura,
    ano_mes_abertura,
    ano_fechamento,
    ano_mes_fechamento,
    id_filial,
    filial,
    id_setor,
    setor,
    id_assunto,
    assunto,
    id_relator,
    relator,
    id_tecnico,
    tecnico,
    SUM(qtd) AS qtd_total,
    SUM(CASE WHEN status = 'Reagendada' THEN qtd ELSE 0 END) AS qtd_reagendada,
    SUM(CASE WHEN status = 'Encaminhada' THEN qtd ELSE 0 END) AS qtd_encaminhada,
    SUM(CASE WHEN status = 'Assumida' THEN qtd ELSE 0 END) AS qtd_assumida,
    SUM(CASE WHEN status = 'Deslocamento' THEN qtd ELSE 0 END) AS qtd_deslocamento,
    SUM(CASE WHEN status = 'Agendada' THEN qtd ELSE 0 END) AS qtd_agendada,
    SUM(CASE WHEN status = 'Aberta' THEN qtd ELSE 0 END) AS qtd_aberta,
    SUM(CASE WHEN status = 'Em Execução' THEN qtd ELSE 0 END) AS qtd_execucao,
    SUM(CASE WHEN status = 'Fechada' THEN qtd ELSE 0 END) AS qtd_fechada,
    AVG(CAST(unix_timestamp(data_fechamento) - unix_timestamp(data_abertura) AS DECIMAL)) AS tempo_medio_fechamento_segundos,
    SUM(CASE WHEN status_sla = 'Dentro do Prazo' THEN qtd ELSE 0 END) AS qtd_dentro_prazo,
    SUM(CASE WHEN status_sla = 'Fora do Prazo' THEN qtd ELSE 0 END) AS qtd_fora_prazo
FROM STATUS_COUNTS
GROUP BY 
    ano_abertura,
    ano_mes_abertura,
    ano_fechamento,
    ano_mes_fechamento,
    id_filial,
    filial,
    id_setor,
    setor,
    id_assunto,
    assunto,
    id_relator,
    relator,
    id_tecnico,
    tecnico
"""

# Executar a query
resultado = spark.sql(query)

# Visualizar os resultados
resultado.show(truncate=False)


AnalysisException: Table or view not found: silver_ordem_servico; line 26 pos 8;
'WithCTE
:- 'CTERelationDef 0, false
:  +- 'SubqueryAlias BASE_PERFORMANCE
:     +- 'Sort ['t1.ano_mes_abertura ASC NULLS FIRST, 't2.id ASC NULLS FIRST], true
:        +- 'Project ['t1.ano_abertura, 't1.ano_mes_abertura, 't1.data_abertura, 't1.ano_fechamento, 't1.ano_mes_fechamento, 't1.data_fechamento, 't2.id AS id_filial#12, 't2.fantasia AS filial#13, 't4.id AS id_setor#14, 't4.setor AS setor#15, 't6.id AS id_relator#16, 't6.login AS relator#17, 't5.id AS id_tecnico#18, 't5.funcionario AS tecnico#19, 't3.id AS id_assunto#20, 't3.assunto AS assunto#21, 't1.id AS ordem_servico_id#22, 't1.id_status, 't1.status, 't1.id_status_sla, 't1.status_sla]
:           +- 'Join LeftOuter, ('t6.id = 't1.id_login)
:              :- 'Join LeftOuter, ('t5.id = 't1.id_tecnico)
:              :  :- 'Join LeftOuter, ('t4.id = 't1.id_setor)
:              :  :  :- 'Join LeftOuter, ('t3.id = 't1.id_assunto)
:              :  : 